In [2]:
import pandas as pd
import numpy as np
from cyvcf2 import VCF
import os

In [4]:
"""set path of VCF file and index the file using tabix"""

vcf_path = '//Volumes/Samsung_T5/ncVEP/VCFs/chr21/gnomad.genomes.v4.1.sites.chr21.vcf.bgz'

# use tabix to index the gzipped VCF file
os.system('tabix -p vcf ' + vcf_path)

vcf = VCF(vcf_path)


In [5]:
#view variant at a specific position
position = 5030082

for variant in vcf('chr21:'+str(position)+'-'+str(position)):
    print(variant)

chr21	5030082	.	G	A	.	AC0;AS_VQSR	AC=0;AN=2;AF=0;AC_XX=0;AF_XX=0;AN_XX=2;nhomalt_XX=0;AC_XY=0;AN_XY=0;nhomalt_XY=0;nhomalt=0;AC_afr_XX=0;AN_afr_XX=0;nhomalt_afr_XX=0;AC_afr_XY=0;AN_afr_XY=0;nhomalt_afr_XY=0;AC_afr=0;AN_afr=0;nhomalt_afr=0;AC_ami_XX=0;AN_ami_XX=0;nhomalt_ami_XX=0;AC_ami_XY=0;AN_ami_XY=0;nhomalt_ami_XY=0;AC_ami=0;AN_ami=0;nhomalt_ami=0;AC_amr_XX=0;AN_amr_XX=0;nhomalt_amr_XX=0;AC_amr_XY=0;AN_amr_XY=0;nhomalt_amr_XY=0;AC_amr=0;AN_amr=0;nhomalt_amr=0;AC_asj_XX=0;AN_asj_XX=0;nhomalt_asj_XX=0;AC_asj_XY=0;AN_asj_XY=0;nhomalt_asj_XY=0;AC_asj=0;AN_asj=0;nhomalt_asj=0;AC_eas_XX=0;AF_eas_XX=0;AN_eas_XX=2;nhomalt_eas_XX=0;AC_eas_XY=0;AN_eas_XY=0;nhomalt_eas_XY=0;AC_eas=0;AF_eas=0;AN_eas=2;nhomalt_eas=0;AC_fin_XX=0;AN_fin_XX=0;nhomalt_fin_XX=0;AC_fin_XY=0;AN_fin_XY=0;nhomalt_fin_XY=0;AC_fin=0;AN_fin=0;nhomalt_fin=0;AC_mid_XX=0;AN_mid_XX=0;nhomalt_mid_XX=0;AC_mid_XY=0;AN_mid_XY=0;nhomalt_mid_XY=0;AC_mid=0;AN_mid=0;nhomalt_mid=0;AC_nfe_XX=0;AN_nfe_XX=0;nhomalt_nfe_XX=0;AC_nfe_XY=0;AN_

In [46]:
# get the header of the VCF file
header = vcf.raw_header
print(header)

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##hailversion=0.2.122-be9d88a80695
##FILTER=<ID=AC0,Description="Allele count is zero after filtering out low-confidence genotypes (GQ < 20; DP < 10; and AB < 0.2 for het calls)">
##FILTER=<ID=AS_VQSR,Description="Failed VQSR filtering thresholds of -2.502 for SNPs and -0.7156 for indels">
##FILTER=<ID=InbreedingCoeff,Description="Inbreeding coefficient < -0.3">
##INFO=<ID=AC,Number=A,Type=Integer,Description="Alternate allele count">
##INFO=<ID=AN,Number=1,Type=Integer,Description="Total number of alleles">
##INFO=<ID=AF,Number=A,Type=Float,Description="Alternate allele frequency">
##INFO=<ID=grpmax,Number=A,Type=String,Description="Genetic ancestry group with the maximum allele frequency">
##INFO=<ID=fafmax_faf95_max,Number=A,Type=Float,Description="Maximum filtering allele frequency (using Poisson 95% CI) across genetic ancestry groups">
##INFO=<ID=fafmax_faf95_max_gen_anc,Number=A,Type=String,Description="Gene

In [49]:
#save only if variant_type is SNV
chrom = []
pos = []
ID = []
ref = []
alt = []
for variant in vcf:
    if variant.INFO["variant_type"] == "snv":
        chrom.append(variant.CHROM)
        pos.append(variant.POS)
        ID.append(variant.ID)
        ref.append(variant.REF)
        alt.append(variant.ALT)

# create a dataframe from the lists
df = pd.DataFrame({'chrom': chrom, 'pos': pos, 'ID': ID, 'ref': ref, 'alt': alt})

#convert alt to string
df['alt'] = df['alt'].apply(lambda x: str(x[0]))

In [71]:
#convert pos to string
df['pos'] = df['pos'].apply(lambda x: str(x))

#if ID is missing, replace with '.'
df['ID'] = df['ID'].apply(lambda x: '.' if x is None else x)

In [72]:
#save bed file
df['annot'] = df['chrom'] + '_' + df['pos'] + ";" +df['ID'] + '_' + df['ref'] + ':' + df['alt']
bed_out = df[['chrom', 'pos', 'pos', 'annot']]
bed_out.to_csv('/Volumes/Samsung_T5/ncVEP/VCFs/gnomad_chr21_snvs.bed', sep='\t', header=False, index=False)